In [3]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


openai client 테스트

In [4]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)

영어를 한국어로 번역하는 5-shot prompt 를 작성하고 "dog"를 번역해보세요.

In [5]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "영어를 한국어로 번역해야합니다."},
    {"role": "system", "content": "dog 는 한국어로 개"},
    {"role": "system", "content": "cat은 한국어로 고양이"},
    {"role": "system", "content": "lion 은 한국어로 사자"},
    {"role": "system", "content": "tiger 는 한국어로 호랑이"},
    {"role": "user", "content": "dog 가 한국어로 뭐야?"}],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

dog는 한국어로 "개"입니다.

영화 리뷰에 대한 sentiment (positive or negative) 를 결정하는 5-shot prompt 를 작성하고 "The storyline was dull and uninspiring." 에 대한 결과를 확인해보세요.

In [6]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "영화 리뷰에 대한 sentiemnt 를 긍정적 혹은 부정적으로 판단해야합니다"},
    {"role": "system", "content": "이 영화는 정말 재미가 없어. 에 대한 리뷰는 부정적입니다."},
    {"role": "system", "content": "이 영화의 스토리는 난해하고 이해하기 어려워. 에 대한 리뷰는 부정적입니다"},
    {"role": "system", "content": "이 영화는 정말 재밌고, 흥미로워. 한번 더 보고싶어 에 대한 리뷰는 긍정적입니다"},
    {"role": "system", "content": "이 영화는 스토리가 탄탄하고 극의 흐름이 자연스러워. 에 대한 리뷰는 긍정적입니다."},
    {"role": "user", "content": "The storyline was dull and uninspiring."}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

부정적입니다.

자연어를 SQL 쿼리로 바꿔주는 few-shot prompt 를 작성해보세요.

In [7]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": """
    Convert the following natural language requests into SQL queries:
1. "employee 의 임금이 50000이 넘는 경우를 조회하는 쿼리": SELECT * FROM employees WHERE salary > 50000;
2. "상품의 재고가 0 인 경우를 조회하는 쿼리": SELECT * FROM products WHERE stock = 0;
3. "수학 점수가 90이 넘는 학생의 이름을 조회하는 쿼리 ": SELECT name FROM students WHERE math_score > 90;
4. "상품의 주문일이 현재 날짜 기준 30일 전인 경우를 조회하는 쿼리 ": SELECT * FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);
5. "각 도시에 살고있는 고객의 수를 조회하는 쿼리": SELECT city, COUNT(*) FROM customers GROUP BY city;
    """},
    {"role": "user", "content": "Find the average salary of employees in the marketing department."}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

To find the average salary of employees in the marketing department, you can use the following SQL query:

```sql
SELECT AVG(salary) FROM employees WHERE department = 'marketing';
```

다음 문제를 푸는 Chain of thought prompt 를 작성해보세요.

In [9]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
    {"role": "user", "content": """ 
        Solve the following problem step-by-step: 23 + 47

Step-by-step solution:
1. 10+50 = 60
2. 13+100 = 113
3. 1235+1235 = 2470
4. 2222+2222 = 4444
    """}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

To solve the problem \(23 + 47\) step-by-step, we can follow these steps:

1. **Align the Numbers**: Write the numbers one above the other, aligning the digits by their place values.
   ```
     23
   + 47
   ```

2. **Add the Units Place**: Start by adding the digits in the rightmost column (the units place).
   - \(3 + 7 = 10\)
   - Write down 0 and carry over 1 to the next column.

3. **Add the Tens Place**: Now, add the digits in the tens place, including any carry from the previous step.
   - \(2 + 4 + 1 = 7\) (the 1 is from the carry)
   
4. **Write Down the Result**: Combine the results of each column.
   ```
     23
   + 47
   -----
     70
   ```

So, \(23 + 47 = 70\).

아래 문제 (Intermediate-1, Intermediate-2) 를 푸는 과정을 담은 CoT 예시를 작성해보세요.

In [10]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
    {"role": "user", "content": """ 
        Solve the following logic puzzle step-by-step:
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.

Determine the favorite color of each friend.

Step-by-step solution:
1. 사람 A 가 빨강을 좋아한다면 사람 B,C 는 빨강을 좋아하지 않습니다.
2. 사람 B 가 파랑을 좋아한다면 사람 A,C 는 파랑을 좋아하지 않습니다.
3. 사람 C 가 초록을 좋아한다면 사람 A,B 는 초록을 좋아하지 않습니다.
4. 가능한 경우를 출력합니다.
    """}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

Let's solve the puzzle step-by-step based on the clues provided.

We have three friends: Alice, Bob, and Carol. Their favorite colors are red, blue, and green, and we are given the following clues:

1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.

Now let's analyze the clues one by one.

**Step 1:** Analyze the clues.

- From clue 1, we know Alice does not like red. Therefore, Alice can either like blue or green.
  
- From clue 2, Bob does not like blue. Therefore, Bob can either like red or green.

- From clue 3, we know Carol likes green. Since Carol's favorite color is green, Alice and Bob cannot like green and must like either red or blue.

**Step 2:** Determine the implications of each clue.

Since we know Carol's favorite color is green:
- Alice cannot like red (because of clue 1), and since Carol likes green, Alice must like blue.
  
- Now, since Alice likes blue, Bob cannot like blue (clue 2) and because Carol likes green, Bob must like red.

At thi

# Intermediate - 2

In [16]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
    {"role": "user", "content": """ 
        Solve the following logic puzzle step-by-step:
Four people (A, B, C, D) are sitting in a row. We know that:
1. A is not next to B.
2. B is next to C.
3. C is not next to D.

Determine the possible seating arrangements.

Step-by-step solution:
1. ABCD 라면, A 다음에 B 가 앉아있고, B 다음에 C 가 앉아있고, C 다음에 D가 앉아있는 경우입니다. 즉 B가 C 다음에 온다면 BC 는 불가능합니다
2. AABB 와 같이, 좌석순서에는 같은 사람이 존재하지 않습니다.
3. 앉은 순서는 각 사람의 이름으로 제공합니다.

    """}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

To solve the logic puzzle step-by-step, let’s analyze the requirements one by one and identify the valid seating arrangements for A, B, C, and D.

### Given Information:
1. A is not next to B.
2. B is next to C.
3. C is not next to D.

### Step-by-Step Breakdown:

1. **Start with the known relationships:**
   - B must be next to C (from statement 2). This means the possible pairs are (B, C) or (C, B).

2. **Consider the seating arrangements:**
   - Let's denote the seating positions as 1, 2, 3, and 4.
   - We can start forming the arrangements based on the pair (B, C).

3. **Possible arrangements involving B and C:**
   - If we treat BC as a block, the arrangements could be:
     - (B, C) could take positions (1, 2) or (2, 3) or (3, 4).

### Let's examine each case:

#### Case 1: B, C in positions (1, 2)
- Arrangement: (B, C, _, _)
- Possible arrangements for A and D:
  - A cannot be in position 2 (next to B) ⇒ A must be in position 3 or 4.
  - If A is in position 3: (B, C, A, D) is a 

claude API 가 되지 않아 OPENAI 로 테스트

프롬프트 생성 요청

In [32]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
    {"role": "user", "content": """ 
        open ai 에서 사용할 프롬프트를 생성해줘.
        아래와 같이 생성하면 된다.
        1. 주번을 정하는 것을 도울 수 있어야해
        2. 각 사람은 albus.star, dante.93, sera.son, jett.jistol, mark.dd, kylie.do, daisy.potter2, moka.gold, rein.withus, seed.sim, muscle.kim, pharrell.mp3 가 있어
        3. 그 중 정 / 부가 존재해서 정과 정끼리는 붙이면 안되고, 2명 페어로 이루어져있어.
        4. 정은 albus.star, dante.93, cooper.lee, kylie.do, rein.withus, seed.sim, pharrell.mp3 야. 부는 sera.son, jett.jistol, mark.dd, daisy.potter2, moka.gold, muscle.kim 야
        5. 주번은 매주 월요일에 교체가 됨.
        6. cooper.lee 는 휴가나 등으로 주번 페어를 만들 수 없는 경우 예비로 투입될 수 있어.
        7. 한 사이클은 정 / 부가 한번씩 배치되면 끝이야. 또한 모든 정 / 부는 한 사이클에 무조건 한번은 주번 페어에 들어가야해.
        8. 만약 한 사이클에 포함되지 않은 사람이 있음 말해줘
        9. 요청에 휴가기록이 있다면 그 주에는 주번 페어를 다른 사람을 배정해야해.
        10. 예를 들어 albus.star 가 2주차에 휴가라면, 휴가가 아닌 다른 주차에 무조건 배정되어야해.

        이랬을 때 open ai 에 적합한 프롬프트를 생성해줘

    """}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

다음은 OpenAI에 적합한 프롬프트입니다:

---

**프롬프트: 주번 페어 배정 시스템**

주번 페어를 구성하는 시스템을 만드세요. 아래 조건을 반드시 준수하세요:

1. 주번은 매주 월요일에 변경되며, 각 주에 2명으로 이루어진 페어로 구성됩니다.
2. 참여자는 다음과 같습니다: 
   - 정: albus.star, dante.93, cooper.lee, kylie.do, rein.withus, seed.sim, pharrell.mp3 
   - 부: sera.son, jett.jistol, mark.dd, daisy.potter2, moka.gold, muscle.kim
3. 정과 정은 같은 페어에 포함될 수 없으며, 부와 부도 마찬가지입니다.
4. 각 사이클에서는 정과 부가 한 번씩 배치되어야 하며, 모든 정/부는 반드시 한 사이클 내에 포함되어야 합니다.
5. cooper.lee는 예비 인원으로, 다른 사람과 페어를 만들 수 없는 경우에만 사용될 수 있습니다.
6. 만약 어느 주에 휴가기록이 있다면, 해당 주에는 주번 페어를 다른 사람으로 배정해야 합니다. 예를 들어, albus.star가 2주차에 휴가인 경우, 다른 주차에 반드시 배정되어야 합니다.
7. 주번 배정 결과를 출력하며, 만약 한 사이클에 포함되지 않은 사람이 있다면 그 사실을 알려주세요.

입력 사항: 주마다 각 사람의 휴가 일정(예: ["albus.star": "2주차", "jett.jistol": "3주차"])을 제공하며, 이를 바탕으로 적절한 주번 페어를 구성하세요.

---

이 프롬프트를 사용하면 OpenAI 모델이 위 조건을 기반으로 하여 주번 페어를 적절하게 할당할 수 있습니다.

In [ ]:
prompt 적용

In [47]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
    {"role": "user", "content": """ 
        ---

**프롬프트: 주번 페어 배정 시스템**

주번 페어를 구성하는 시스템을 만드세요. 아래 조건을 반드시 준수하세요:

1. 주번은 매주 월요일에 변경되며, 각 주에 정 1명, 부 1명으로 구성됩니다.
2. 참여자는 다음과 같습니다: 
   - 정: albus.star, dante.93, kylie.do, rein.withus, seed.sim, pharrell.mp3 
   - 부: sera.son, jett.jistol, mark.dd, daisy.potter2, moka.gold, muscle.kim
3. 정과 정은 같은 페어에 포함될 수 없으며, 부와 부도 마찬가지입니다. 예를 들어 albus.star 와 dante.93 은 같이 배정이 안됩니다.
4. 각 사이클에서는 정과 부가 한 번씩 배치되어야 하며, 모든 정/부는 반드시 한 사이클 내에 포함되어야 합니다.
5. cooper.lee는 예비 인원으로, 다른 사람과 페어를 만들 수 없는 경우에만 사용될 수 있습니다.
6. 만약 어느 주에 휴가기록이 있다면, 해당 주에는 주번 페어를 다른 사람으로 배정해야 합니다. 예를 들어, albus.star가 2주차에 휴가인 경우, albus.star 는 다른 주차에 반드시 배정되어야 합니다.
7. 주번 배정 결과를 출력하며, 만약 한 사이클에 포함되지 않은 사람이 있다면 그 사실을 알려주세요.
8. 각 정과 각 부는 무조건 한사이클에 최소 한번은 포함되어야합니다.

---

albus.star 는 1주차에 휴가, jett.jistol 은 2주차에 휴가일 때, 
이를 바탕으로 6주치의 적절한 주번 페어를 구성해줄 수 있어?

    """}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

주번 페어 배정 시스템에 따라 주번을 배정하고, 주차 및 휴가 정보를 반영해 6주 치의 주번 페어를 구성하겠습니다.

### 참여자 목록

- **정**: albus.star, dante.93, kylie.do, rein.withus, seed.sim, pharrell.mp3 
- **부**: sera.son, jett.jistol, mark.dd, daisy.potter2, moka.gold, muscle.kim

### 휴가 정보
- 1주차: albus.star (정)
- 2주차: jett.jistol (부)

### 주번 페어 구성 (6주)

| 주차 | 정         | 부            |
|------|------------|---------------|
| 1    | dante.93  | sera.son      |
| 2    | kylie.do  | mark.dd       |
| 3    | albus.star | jett.jistol   |
| 4    | rein.withus| daisy.potter2 |
| 5    | seed.sim   | moka.gold     |
| 6    | pharrell.mp3| muscle.kim   |

### 정리
1. 1주차: albus.star가 휴가이므로 dante.93와 sera.son을 배정. 
2. 2주차: jett.jistol가 휴가이므로 kylie.do와 mark.dd를 배정.
3. 3주차: albus.star가 1주차에 배정되지 않아 jett.jistol과 페어 형성.
4. 각 주차마다 정과 부가 반드시 한 번씩 배치되며, 주마다 정과 부가 서로 다른 조합으로 구성됨.
5. 모든 정과 부는 최소 한 사이클에 포함됨.

### 확인 사항
- 모든 참여자는 적어도 한 번씩 주번으로 배정됨.
  
### 결론
대상자 누구도 사이클 내에 포함되지 않는 일이 없었으며, 따로 예비 인원 cooper.lee는 사용되지 않았습니다.